# Initialize Connection Weights

After a neural network is constructed, the connection weights of different layers should be initialized. The ``brainpy.training.initialize`` module provides some build-in initializers. Besides, users can customize the initializers of their own. Here we will briefly talk about how to use these build-in initializers and how to create customized initializers.

**Contents**

- [General structure of initializers](#General-structure-of-initializers)
- [Build-in regular initializers](#Build-in-regular-initializers)
- [Build-in random initializers](#Build-in-random-initializers)
- [Build-in decay initializers](#Build-in-decay-initializers)
- [Customize your initializers](#Customize-your-initializers)

## General structure of initializers

### Base Class: bp.init.Initializer

Initializers initialize the coonection weights between two groups of neurons (though they can refer to the same group). The base class of initializers are [`brainpy.training.initialize.Initializer`](../apis/auto/training/initialize.rst), which can be accessed by the shortcut `bp.init`. All initializers, build-in or costumized, should inherit the `Initializer` class. 

Weight initialization is implemented in the `__call__` function, so that it can be realized automatically when the initializer is called. The `__call__` function has a `shape` parameter that has different meanings in the following two superclasses and returns a matrix containing initialized weights.

In [2]:
import brainpy as bp
# bp.training.initialize can be accessed by bp.init

ImportError: ImportError: DLL load failed while importing _fblas: 找不到指定的模块。

At:
  D:\Users\XiaoyuChen\anaconda3\lib\site-packages\scipy\linalg\blas.py(213): <module>
  <frozen importlib._bootstrap>(219): _call_with_frames_removed
  <frozen importlib._bootstrap_external>(783): exec_module
  <frozen importlib._bootstrap>(686): _load_unlocked
  <frozen importlib._bootstrap>(975): _find_and_load_unlocked
  <frozen importlib._bootstrap>(991): _find_and_load
  D:\Users\XiaoyuChen\anaconda3\lib\site-packages\scipy\linalg\misc.py(3): <module>
  <frozen importlib._bootstrap>(219): _call_with_frames_removed
  <frozen importlib._bootstrap_external>(783): exec_module
  <frozen importlib._bootstrap>(686): _load_unlocked
  <frozen importlib._bootstrap>(975): _find_and_load_unlocked
  <frozen importlib._bootstrap>(991): _find_and_load
  D:\Users\XiaoyuChen\anaconda3\lib\site-packages\scipy\linalg\__init__.py(195): <module>
  <frozen importlib._bootstrap>(219): _call_with_frames_removed
  <frozen imp

### Superclass 1: bp.init.InterLayerInitializer

The [`InterLayerInitializer`](../apis/auto/training/initialize.rst) is an abstract subclass of `Initializer`. Subclasses of `InterLayerInitializer` initialize the weights between two fully connected layers. The `shape` parameter of the `__call__` function should be a 2-element tuple $(m, n)$, which refers to the number of presynaptic neurons $m$ and of postsynaptic neurons $n$. The output of the `__call__` function is a `bp.math.ndarray` with the shape of $(m, n)$, where the value at $(i, j)$ is the initialized weight of the presynaptic neuron $i$ to postsynpatic neuron $j$.

### Superclass 2: bp.init.IntraLayerInitializer

The [`IntraLayerInitializer`](../apis/auto/training/initialize.rst) is also an abstract subclass of `Initializer`. Subclasses of `InterLayerInitializer` initialize the weights within a layer. The `shape` parameter of the `__call__` function refers to the the structure of the neural population $(n_1, n_2, ..., n_d)$. The `__call__` function returns a 2-D `bp.math.ndarray` with the shape of $(\prod_{k=1}^d n_k, \prod_{k=1}^d n_k)$. In the 2-D array, the value at $(i, j)$ is the initialized weight of neuron $i$ to neuron $j$ of the flattened neural sequence.

## Build-in regular initializers

Regular initializers all belong to `InterLayerInitializer` and initialize the connection weights between two layers with a regular pattern. There are `ZeroInit`, `OneInit`, and `Identity` initializers in build-in regular initializers. Here we show how to use the `OneInit` initializer. The remaining two classes are used in a similar way. See [Weight Initialization](../apis/auto/training/initialize.rst) for more details.

### bp.init.OneInit

This initializer initializes all the weights with the same given value. In the following example, 5 presynaptic neurons are fully connected to 6 postsynaptic neurons with the same weight value:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# visualization
def mat_visualize(matrix, cmap=plt.cm.get_cmap('coolwarm')):
    im = plt.matshow(matrix, cmap=cmap)
    plt.colorbar(mappable=im, shrink=0.8, aspect=15)
    plt.show()

In [ ]:
shape = (5, 6)
one_init = bp.init.OneInit(value=2.5)
weights = one_init(shape)
print(weights)

## Build-in random initializers

Random initializers all belong to `InterLayerInitializer` and initialize the connection weights between two layers with a random distribution. There are `Normal`, `Uniform`, `Orthogonal` and other initializers in build-in regular initializers. Here we show how to use the `Normal` and `Uniform` initializer. See [Weight Initialization](../apis/auto/training/initialize.rst) for more details.

### bp.init.Normal

This initializer initializes the weights with a normal distribution. The variance of the distribution changes according to the `scale` parameter. In the following example, 10 presynaptic neurons are fully connected to 20 postsynaptic neurons with the same weight value:

In [ ]:
shape = (10, 20)
normal_init = bp.init.Normal(scale=1.0)
weights = normal_init(shape)
mat_visualize(weights)

### bp.init.Uniform

This initializer initializes the weights with a uniform distribution. The variance of the distribution changes according to the `scale` parameter. In the following example, 10 presynaptic neurons are fully connected to 20 postsynaptic neurons with the same weight value:

In [ ]:
shape = (10, 20)
uniform_init = bp.init.Uniform(min_val=0., max_val=1.)
weights = uniform_init(shape)
mat_visualize(weights)

## Build-in decay initializers

Decay initializers all belong to `IntraLayerInitializer` and initialize the connection weights within a layer with a decay function according to the neural distance. There are `GaussianDecay` and `DOGDecay` initializers in build-in decay initializers. Below are examples of how to use them.

### brainpy.training.initialize.GaussianDecay

This initializer builds a Gaussian connectivity pattern within a population of neurons, where the weights decay with a gaussian function.
Specifically, for any pair of neurons $ (i, j) $, the weight is computed as

$$
w(i, j) = w_{max} \cdot \exp(-\frac{\sum_{k=1}^n |v_k^i - v_k^j|^2 }{2\sigma^2})
$$

where $ v_k^i $ is the $ i $-th neuron's encoded value (position) at dimension $ k $.

The example below is a neural population with the size of $ 5 \times 5 $. Note that this shape is the structure of the target neural population, not the size of presynaptic and postsynaptic neurons.

In [ ]:
size = (5, 5)
gaussian_init = bp.init.GaussianDecay(sigma=2., max_w=10., include_self=True)
weights = gaussian_init(size)
print('shape of weights: {}'.format(weights.shape))

Self-connections are created if `include_self=True`. The connection weights of each neuron with others are stored in each row of `weights`. For instance, the connection weights of neuron(0, 0) to other neurons are stored in `weights[0]`. After reshaping, the weights are:

In [ ]:
mat_visualize(weights[0].reshape(size), cmap=plt.cm.get_cmap('Reds'))

### brainpy.training.initialize.DOGDecay

This initializer builds a Difference-Of-Gaussian (DOG) connectivity pattern within a population of neurons. Specifically, for the given pair of neurons $ (i, j) $, the weight between them is computed as

$$
w(i, j) = w_{max}^+ \cdot \exp(-\frac{\sum_{k=1}^n |v_k^i - v_k^j|^2}{2\sigma_+^2}) - w_{max}^- \cdot \exp(-\frac{\sum_{k=1}^n |v_k^i - v_k^j|^2}{2\sigma_-^2})
$$

where $ v_k^i $ is the $ i $-th neuron's encoded value (position) at dimension $ k $.

<img src="../_static/DOGDecay.png" width="200 px">

The example below is a neural population with the size of $ 10 \times 10 $:

In [ ]:
size = (10, 10)
gaussian_init = bp.init.DOGDecay(sigmas=(1., 3.), max_ws=(10., 5.), min_w=0.1, include_self=True)
weights = gaussian_init(size)
print('shape of weights: {}'.format(weights.shape))

Weights smaller than `min_w` will not be created. `min_w` $ = 0.005 \times min( $ `max_ws` $ ) $ if it is not assigned with a value.
The organization of `weights` is similar to that in the `GaussianDecay` initializer. For instance, the connection weights of neuron (3, 4) to other neurons after reshaping are shown as below:

In [ ]:
mat_visualize(weights[34].reshape(size), cmap=plt.cm.get_cmap('Reds'))

## Customize your initializers

BrainPy also allows users to customize the weight initializers of their own. When customizing a initializer, users should follow the instructions below:
- Your initializer should inherit `brainpy.training.initialize.Initializer`.
- Override the `__call__` funtion, to which the `shape` parameter should be given.

Here is an example of creating an inter-layer initializer that initialize the weights as follows:

$$
w(i, j) = max(w_{max} - \sigma |v_i - v_j|, 0)
$$

In [ ]:
class LinearDecay(bp.init.InterLayerInitializer):
    def __init__(self, max_w, sigma=1.):
        self.max_w = max_w
        self.sigma = sigma
    
    def __call__(self, shape, dtype=None):
        mat = bp.math.zeros(shape, dtype=dtype)
        n_pre, n_post = shape
        seq = np.arange(n_pre)
        current_w = self.max_w
        
        for i in range(max(n_pre, n_post)):
            if current_w <= 0:
                break
            seq_plus = ((seq + i) >= 0) & ((seq + i) < n_post)
            seq_minus = ((seq - i) >= 0) & ((seq - i) < n_post)
            mat[seq[seq_plus], (seq + i)[seq_plus]] = current_w
            mat[seq[seq_minus], (seq - i)[seq_minus]] = current_w
            current_w -= self.sigma
        
        return mat

In [ ]:
shape = (10, 15)
lin_init = LinearDecay(max_w=5, sigma=1.)
weights = lin_init(shape)
mat_visualize(weights, cmap=plt.cm.get_cmap('Reds'))

---

- Xiaoyu Chen (c-xy17@tsinghua.org.cn)
- Update at 2021.12.02

---